This notebook was based on someone else's code which I downloaded and edited, but I can't find the original kernel.  I also took a lot from Grzegorz Sionkowski's Bayesian Optimization kernel, and used a modified version of that kernel to tune the parameters.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import math

from trackml.dataset import load_event, load_dataset
from trackml.score import score_event

from multiprocessing import Pool

def create_one_event_submission(event_id, hits, labels):
    sub_data = np.column_stack(([event_id]*len(hits), hits.hit_id.values, labels))
    submission = pd.DataFrame(data=sub_data, columns=["event_id", "hit_id", "track_id"]).astype(int)
    return submission

In [ ]:
# Change this according to your directory preferred setting
path_to_train = "../input/train_1/"
path_to_test = "../input/test/"

In [ ]:
from sklearn.preprocessing import StandardScaler
import hdbscan
from scipy import stats
from tqdm import tqdm_notebook as tqdm
from sklearn.cluster import DBSCAN, AgglomerativeClustering

class Clusterer(object):
    def __init__(self,rz_scales=[0.65, 0.965, 1.528], eps=0.0035, dz0 = -0.00070, stepdz = 0.00001, stepeps = 0.000005, num_loops=100):                        
        self.rz_scales=rz_scales
        self.epsilon = eps
        self.dz0 = dz0
        self.stepdz = stepdz
        self.stepeps = stepeps
        self.num_loops = num_loops
        
    # remove outliers
    def _eliminate_outliers(self,labels,M):
        norms=np.zeros((len(labels)),np.float32)
        indices=np.zeros((len(labels)),np.float32)
        for i, cluster in tqdm(enumerate(labels),total=len(labels)):
            if cluster == 0:
                continue
            index = np.argwhere(self.clusters==cluster)
            index = np.reshape(index,(index.shape[0]))
            indices[i] = len(index)
            x = M[index]
            norms[i] = self._test_quadric(x)
        threshold1 = np.percentile(norms,90)*5
        threshold2 = 25
        threshold3 = 6
        for i, cluster in enumerate(labels):
            if norms[i] > threshold1 or indices[i] > threshold2 or indices[i] < threshold3:
                self.clusters[self.clusters==cluster]=0   
    
    # not sure what this function does?
    def _test_quadric(self,x):
        if x.size == 0 or len(x.shape)<2:
            return 0
        Z = np.zeros((x.shape[0],10), np.float32)
        Z[:,0] = x[:,0]**2
        Z[:,1] = 2*x[:,0]*x[:,1]
        Z[:,2] = 2*x[:,0]*x[:,2]
        Z[:,3] = 2*x[:,0]
        Z[:,4] = x[:,1]**2
        Z[:,5] = 2*x[:,1]*x[:,2]
        Z[:,6] = 2*x[:,1]
        Z[:,7] = x[:,2]**2
        Z[:,8] = 2*x[:,2]
        Z[:,9] = 1
        v, s, t = np.linalg.svd(Z,full_matrices=False)        
        smallest_index = np.argmin(np.array(s))
        T = np.array(t)
        T = T[smallest_index,:]        
        norm = np.linalg.norm(np.dot(Z,T), ord=2)**2
        return norm

    # standard scale our data
    def _preprocess(self, hits):
        x = hits.x.values
        y = hits.y.values
        z = hits.z.values

        r = np.sqrt(x**2 + y**2 + z**2)
        hits['x2'] = x/r
        hits['y2'] = y/r

        r = np.sqrt(x**2 + y**2)
        hits['z2'] = z/r

        ss = StandardScaler()
        X = ss.fit_transform(hits[['x2', 'y2', 'z2']].values)
        for i, rz_scale in enumerate(self.rz_scales):
            X[:,i] = X[:,i] * rz_scale
       
        return X
    
    def _init(self,dfh):
        dfh['s1'] =dfh.hit_id
        dfh['N1'] =1
        dfh['r'] = dfh['r'] = np.sqrt(dfh['x'].values**2+dfh['y'].values**2+dfh['z'].values**2)
        dfh['rt'] = np.sqrt(dfh['x'].values**2+dfh['y'].values**2)
        dfh['a0'] = np.arctan2(dfh['y'].values,dfh['x'].values)
        dfh['z1'] = dfh['z'].values/dfh['rt'].values
        dfh['z2'] = dfh['z']/dfh['r']
        dfh['z3'] = 1/dfh['z1'].values
        mm = 1

        for ii in range(self.num_loops):
            mm = mm*(-1)                               
            dfh['a1'] = dfh['a0'].values+mm*(dfh['rt']+self.stepeps*dfh['rt']**2)/1000*(ii/2)/180*math.pi
            dfh['z'].values*np.sign(dfh['z'].values)
            dfh['x1'] = dfh['a1'].values/dfh['z1'].values
            dfh['sina1'] = np.sin(dfh['a1'].values)
            dfh['cosa1'] = np.cos(dfh['a1'].values)
            ss = StandardScaler()
            dfs = ss.fit_transform(dfh[['sina1','cosa1','z1','z2', 'z3', 'x1']].values)
            cx = np.array([1.08, 1.08, .38, .25, 0.009, 0.0001])
            dfs = np.multiply(dfs, cx)

            clusters=DBSCAN(eps=self.epsilon,min_samples=1,metric='euclidean',n_jobs=1).fit(dfs).labels_ 

            if ii==0:
                dfh['s1'] = clusters
                dfh['N1'] = dfh.groupby('s1')['s1'].transform('count')

            # else update our hits conditionally, if it's a better fit
            else:
                # put our new clusters to another feature
                dfh['s2'] = clusters

                # get the count of those clusters
                dfh['N2'] = dfh.groupby('s2')['s2'].transform('count')
                maxs1 = dfh['s1'].max()

                # if our new clusters are bigger, but less than 20, use the new ones instead
                cond = np.where((dfh['N2'].values>dfh['N1'].values) & (dfh['N2'].values<20))
                s1 = dfh['s1'].values
                s1[cond] = dfh['s2'].values[cond]+maxs1

                # write the new clusters back to our dataframe
                dfh['s1'] = s1
                dfh['s1'] = dfh['s1'].astype('int64')
                dfh['N1'] = dfh.groupby('s1')['s1'].transform('count')
        
        # return our clusters
        return dfh['s1'].values    
    
    def predict(self, hits):    
        # init our clusters
        self.clusters = self._init(hits) 
        
        # preprocess our data
        X = self._preprocess(hits) 
        
        # create our last clusterer
        cl = hdbscan.HDBSCAN(min_samples=1,min_cluster_size=7, metric='braycurtis',cluster_selection_method='leaf',algorithm='best', leaf_size=50)
            
        # labels = unique clusters
        labels = np.unique(self.clusters)
        
        # remove outliers
        self._eliminate_outliers(labels,X)
        
        # init n_labels
        n_labels = 0
        
        # now we loop through the points that haven't been assigned to a cluster and assign them with
        # HDBSCAN
        while n_labels < len(labels):
            n_labels = len(labels)
            max_len = np.max(self.clusters)
            mask = self.clusters == 0
            self.clusters[mask] = cl.fit_predict(X[mask])+max_len
            
        return self.clusters

In [ ]:
eps = 0.0035
dz0 = -0.000720
stepdz = 3e-6
stepeps = 2e-16

def one_loop(event_id):
    hits  = pd.read_csv(path_to_test + '/event%s-hits.csv'%event_id)
    cells = pd.read_csv(path_to_test + '/event%s-cells.csv'%event_id)
    print('Event ID: ', event_id)
                
    # Track pattern recognition 
    model = Clusterer(eps=eps, dz0=dz0, stepdz=stepdz, stepeps=stepeps)
    labels = model.predict(hits)

    # Prepare submission for an event
    one_submission = create_one_event_submission(event_id, hits, labels)
            
    return one_submission

def create_test_submissions(path_to_test = "../input/test", start=0, end=15):
    event_ids = [ '%09d'%i for i in range(start,end) ]

    pool = Pool(processes=4)
    results = pool.map(one_loop, event_ids)
    pool.close()
    
    return results

In [ ]:
results = create_test_submissions(start=0)

In [ ]:
submission = pd.concat(results, axis=0)
submission.to_csv('20180707_dbscan_outliers_agg.csv.gz', index=False, compression='gzip')
print(len(submission))